# detect pattern

In [1]:
import math
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def detect_patterns1(pt1, pt2, win_size, data, time, plot=0):
    # Store start and end point
    pt1_, pt2_ = pt1, pt2

    limit = int(math.floor(data.size / win_size))
    flag = np.zeros([data.size, 1])
    event_flags = np.zeros([limit, 1])
    segments_sd = []

    # Compute standard deviation for each window
    for i in range(limit):
        sub_data = data[pt1:pt2]
        segments_sd.append(np.std(sub_data, ddof=1))
        pt1 = pt2
        pt2 += win_size

    # Compute MAD
    mad = np.sum(np.abs(segments_sd - np.mean(segments_sd))) / len(segments_sd)
    thresh1, thresh2 = 15, 2 * mad

    # Reset pt1 and pt2
    pt1, pt2 = pt1_, pt2_

    # Classify each window
    for j in range(limit):
        std_fos = np.around(segments_sd[j])
        if std_fos < thresh1:
            flag[pt1:pt2] = 3  # No movement
            event_flags[j] = 3
        elif std_fos > thresh2:
            flag[pt1:pt2] = 2  # Movement
            event_flags[j] = 2
        else:
            flag[pt1:pt2] = 1  # Sleeping
            event_flags[j] = 1
        pt1 = pt2
        pt2 += win_size

    # Plotting
    if plot == 1:
        data_for_plot = data
        width = np.min(data_for_plot)
        height = np.max(data_for_plot) + abs(width) if width < 0 else np.max(data_for_plot)

        current_axis = plt.gca()
        plt.plot(np.arange(data.size), data_for_plot, '-k', linewidth=1)
        plt.xlabel('Time [Samples]')
        plt.ylabel('Amplitude [mV]')
        plt.gcf().autofmt_xdate()

        for j in range(limit):
            start_idx = j * win_size
            end_idx = start_idx + win_size
            sub_data = data_for_plot[start_idx:end_idx]
            sub_time = np.arange(start_idx, end_idx) / 50.0

            if event_flags[j] == 3:  # No-movement
                plt.plot(sub_time, sub_data, '-k', linewidth=1)
                current_axis.add_patch(Rectangle((start_idx, width), win_size, height, facecolor="#FAF0BE", alpha=.2))
            elif event_flags[j] == 2:  # Movement
                plt.plot(sub_time, sub_data, '-k', linewidth=1)
                current_axis.add_patch(Rectangle((start_idx, width), win_size, height, facecolor="#FF004F", alpha=1.0))
            else:  # Sleeping
                plt.plot(sub_time, sub_data, '-k', linewidth=1)
                current_axis.add_patch(Rectangle((start_idx, width), win_size, height, facecolor="#00FFFF", alpha=.2))

        plt.savefig('./results/rawData.png')

    # Identify indices to remove (movement and no-movement)
    bad_indices = []
    for j in range(limit):
        start_idx = j * win_size
        end_idx = start_idx + win_size
        if event_flags[j] == 2 or event_flags[j] == 3:
            bad_indices.extend(range(start_idx, end_idx))

    bad_indices = np.array(bad_indices)
    valid_indices = bad_indices[(bad_indices >= 0) & (bad_indices < len(time))]  # Ensure safe indexing


    # Mask out the bad indices
    mask = np.ones(len(data), dtype=bool)
    mask[valid_indices] = False
    filtered_data = data[mask]
    filtered_time = time[mask]

    return filtered_data, filtered_time


In [ ]:
# corroletion 0.19

In [ ]:
import math
import os
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

from band_pass_filtering import band_pass_filtering
from compute_vitals import vitals
from detect_apnea_events import apnea_events
from modwt_matlab_fft import modwt
from modwt_mra_matlab_fft import modwtmra
from remove_nonLinear_trend import remove_nonLinear_trend
from data_subplot import data_subplot
from beat_to_beat import compute_rate

# Main program starts here
print('\nstart processing ...')

# Define sampling frequency
target_fs = 50  # Hz, as the data is already resampled to 50 Hz

# Load BCG and ECG data
bcg_file = r"c:\Users\maram\Downloads\code\code\trial_output\rounded_unix_bcg_synchronized.csv"
ecg_file_path = r"C:\Users\maram\Downloads\code\code\trial_output\ecg_synchronized.csv"

# Load BCG data
bcg_data = pd.read_csv(bcg_file, sep=",", header=None, skiprows=1).values
bcg_data_stream = bcg_data[:, 0].astype(float)  # First column is the signal data (amplitude)
bcg_timestamps = bcg_data[:, 1].astype(np.int64) * 1000  # Convert seconds to milliseconds

# Debug: Print a few BCG timestamps to verify format
print("Sample BCG timestamps (Unix ms):", bcg_timestamps[:5])
print("Sample BCG signal data:", bcg_data_stream[:5])

# Load ECG data (assuming format: Timestamp, Heart Rate, RR Interval in seconds)
ecg_data = pd.read_csv(ecg_file_path, sep=",", header=None, skiprows=1).values
ecg_timestamps_str = ecg_data[:, 0]  # Timestamps as strings
ecg_hr = ecg_data[:, 1].astype(float)  # Heart Rate (reference HR)
ecg_rr = ecg_data[:, 2].astype(float)  # RR Interval in seconds

# Convert ECG timestamps to Unix timestamps (milliseconds)
try:
    ecg_timestamps = pd.to_datetime(ecg_timestamps_str, format='%Y/%m/%d %H:%M:%S').astype('int64') // 10**6  # Convert to ms
except ValueError as e:
    print("Error parsing ECG timestamps:", e)
    print("Please verify the timestamp format in the ECG data.")
    raise

# Debug: Print converted timestamps
print("Sample ECG timestamps (converted to Unix ms):", ecg_timestamps[:5])
print(f"Original ECG data points: {len(ecg_hr)}")
print(f"Original BCG data points: {len(bcg_data_stream)}")

# Detect patterns and remove segments from BCG
start_point, end_point, window_shift = 0, 500, 500
filtered_bcg, filtered_bcg_time = detect_patterns1(start_point, end_point, window_shift, bcg_data_stream, bcg_timestamps, plot=1)

# Find unique seconds in filtered BCG timestamps (converting from ms to s)
filtered_bcg_seconds = np.unique(filtered_bcg_time // 1000)

# Create a mask for ECG data that matches with valid BCG seconds
ecg_seconds = ecg_timestamps // 1000
ecg_mask = np.isin(ecg_seconds, filtered_bcg_seconds)

# Apply the mask to filter ECG data
filtered_ecg_hr = ecg_hr[ecg_mask]
filtered_ecg_timestamps = ecg_timestamps[ecg_mask]
filtered_ecg_rr = ecg_rr[ecg_mask]

print(f"Filtered ECG data points: {len(filtered_ecg_hr)}")
print(f"Filtered BCG data points: {len(filtered_bcg)}")
print(f"Removed {len(ecg_hr) - len(filtered_ecg_hr)} ECG data points ({(1 - len(filtered_ecg_hr)/len(ecg_hr))*100:.2f}%)")

# BCG signal extraction
movement = band_pass_filtering(filtered_bcg, target_fs, "bcg")

# Respiratory signal extraction
breathing = band_pass_filtering(filtered_bcg, target_fs, "breath")
breathing = remove_nonLinear_trend(breathing, 3)
breathing = savgol_filter(breathing, 11, 3)

# Wavelet transform for BCG signal
w = modwt(movement, 'bior3.9', 4)
dc = modwtmra(w, 'bior3.9')
wavelet_cycle = dc[4]

# Vital Signs estimation (10 seconds window)
t1, t2, window_length, window_shift = 0, 500, 500, 500
hop_size = math.floor((window_length - 1) / 2)
limit = int(math.floor(breathing.size / window_shift))

# Compute Heart Rate from BCG (using J-peaks)
beats = vitals(t1, t2, window_shift, limit, wavelet_cycle, filtered_bcg_time, mpd=15, plot=0)
print('\nHeart Rate Information (BCG)')
print('Minimum pulse : ', np.around(np.min(beats)))
print('Maximum pulse : ', np.around(np.max(beats)))
print('Average pulse : ', np.around(np.mean(beats)))
print('length of beats :',len(beats))
# Print filtered data lengths
print(f"Length of filtered ECG HR data: {len(filtered_ecg_hr)}")
print(f"Length of filtered ECG RR data: {len(filtered_ecg_rr)}")
print(f"Length of filtered ECG Timestamps: {len(filtered_ecg_timestamps)}")
print(f"Length of filtered BCG signal: {len(filtered_bcg)}")
print(f"Length of filtered BCG timestamps: {len(filtered_bcg_time)}")

# Breathing Rate with adjusted mpd
breaths = vitals(t1, t2, window_shift, limit, breathing, filtered_bcg_time, mpd=75, plot=0)
print('\nRespiratory Rate Information')
print('Minimum breathing : ', np.around(np.min(breaths)))
print('Maximum breathing : ', np.around(np.max(breaths)))
print('Average breathing : ', np.around(np.mean(breaths)))

# Apnea Events Detection
thresh = 0.3
events = apnea_events(breathing, filtered_bcg_time, thresh=thresh)

# Plot Vitals Example
t1, t2 = 2500, 2500 * 2
data_subplot(filtered_bcg, movement, breathing, wavelet_cycle, t1, t2)

# Compare BCG-derived HR with ECG-derived HR
min_length = min(len(beats), len(filtered_ecg_hr))
bcg_hr = beats[:min_length]
ecg_hr_ref = filtered_ecg_hr[:min_length]

# Compute error metrics
mae = np.mean(np.abs(bcg_hr - ecg_hr_ref))
rmse = np.sqrt(np.mean((bcg_hr - ecg_hr_ref) ** 2))
mape = np.mean(np.abs((bcg_hr - ecg_hr_ref) / ecg_hr_ref)) * 100

print('\nHeart Rate Comparison Metrics')
print('Mean Absolute Error (MAE): ', np.around(mae, 2))
print('Root Mean Square Error (RMSE): ', np.around(rmse, 2))
print('Mean Absolute Percentage Error (MAPE): ', np.around(mape, 2), '%')

# Bland-Altman Plot
mean_hr = (bcg_hr + ecg_hr_ref) / 2
diff_hr = bcg_hr - ecg_hr_ref
mean_diff = np.mean(diff_hr)
std_diff = np.std(diff_hr)
plt.figure(figsize=(8, 6))
plt.scatter(mean_hr, diff_hr, c='blue', alpha=0.5)
plt.axhline(mean_diff, color='red', linestyle='--')
plt.axhline(mean_diff + 1.96 * std_diff, color='gray', linestyle='--')
plt.axhline(mean_diff - 1.96 * std_diff, color='gray', linestyle='--')
plt.xlabel('Mean Heart Rate (BPM)')
plt.ylabel('Difference (BCG HR - ECG HR) (BPM)')
plt.title('Bland-Altman Plot')
plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_bland_altman.png')
plt.close()

# Pearson Correlation Plot
plt.figure(figsize=(8, 6))
plt.scatter(ecg_hr_ref, bcg_hr, c='blue', alpha=0.5)
plt.plot([min(ecg_hr_ref), max(ecg_hr_ref)], [min(ecg_hr_ref), max(ecg_hr_ref)], 'r--')
plt.xlabel('ECG Heart Rate (BPM)')
plt.ylabel('BCG Heart Rate (BPM)')
plt.title('Pearson Correlation Plot')
corr, _ = pearsonr(ecg_hr_ref, bcg_hr)
plt.text(min(ecg_hr_ref), max(bcg_hr), f'Correlation: {corr:.2f}')
plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_pearson_correlation.png')
plt.close()

# Boxplot
plt.figure(figsize=(8, 6))
plt.boxplot([ecg_hr_ref, bcg_hr], labels=['ECG HR', 'BCG HR'])
plt.ylabel('Heart Rate (BPM)')
plt.title('Boxplot of ECG vs BCG Heart Rate')
plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_boxplot.png')
plt.close()

print('\nEnd processing ...')



start processing ...
Sample BCG timestamps (Unix ms): [1699125140000 1699125140000 1699125140000 1699125140000 1699125140000]
Sample BCG signal data: [ -32.7 -284.2 -844.  -242.8 -123.8]
Sample ECG timestamps (converted to Unix ms): Index([1699125140000, 1699125140000, 1699125141000, 1699125142000,
       1699125142000],
      dtype='int64')
Original ECG data points: 12617
Original BCG data points: 520580
Filtered ECG data points: 11896
Filtered BCG data points: 487080
Removed 721 ECG data points (5.71%)

Heart Rate Information (BCG)
Minimum pulse :  60.0
Maximum pulse :  113.0
Average pulse :  87.0
length of beats : 974
Length of filtered ECG HR data: 11896
Length of filtered ECG RR data: 11896
Length of filtered ECG Timestamps: 11896
Length of filtered BCG signal: 487080
Length of filtered BCG timestamps: 487080
Number of filtered ECG data points in the first 10-second window: 16

Respiratory Rate Information
Minimum breathing :  17.0
Maximum breathing :  40.0
Average breathing :  2

corr0.15

In [8]:
import math
import os
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter, butter, filtfilt
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.interpolate import interp1d

# Assuming these are custom functions from your codebase
from band_pass_filtering import band_pass_filtering
from compute_vitals import vitals
from detect_apnea_events import apnea_events
from modwt_matlab_fft import modwt
from modwt_mra_matlab_fft import modwtmra
from remove_nonLinear_trend import remove_nonLinear_trend
from data_subplot import data_subplot
from beat_to_beat import compute_rate

# Custom function detect_patterns1 (included as provided)
def detect_patterns1(pt1, pt2, win_size, data, time, plot=0):
    pt1_, pt2_ = pt1, pt2
    limit = int(math.floor(data.size / win_size))
    flag = np.zeros([data.size, 1])
    event_flags = np.zeros([limit, 1])
    segments_sd = []

    for i in range(limit):
        sub_data = data[pt1:pt2]
        segments_sd.append(np.std(sub_data, ddof=1))
        pt1 = pt2
        pt2 += win_size

    mad = np.sum(np.abs(segments_sd - np.mean(segments_sd))) / len(segments_sd)
    thresh1, thresh2 = 15, 2 * mad
    pt1, pt2 = pt1_, pt2_

    for j in range(limit):
        std_fos = np.around(segments_sd[j])
        if std_fos < thresh1:
            flag[pt1:pt2] = 3  # No movement
            event_flags[j] = 3
        elif std_fos > thresh2:
            flag[pt1:pt2] = 2  # Movement
            event_flags[j] = 2
        else:
            flag[pt1:pt2] = 1  # Sleeping
            event_flags[j] = 1
        pt1 = pt2
        pt2 += win_size

    if plot == 1:
        data_for_plot = data
        width = np.min(data_for_plot)
        height = np.max(data_for_plot) + abs(width) if width < 0 else np.max(data_for_plot)
        current_axis = plt.gca()
        plt.plot(np.arange(data.size), data_for_plot, '-k', linewidth=1)
        plt.xlabel('Time [Samples]')
        plt.ylabel('Amplitude [mV]')
        plt.gcf().autofmt_xdate()

        for j in range(limit):
            start_idx = j * win_size
            end_idx = start_idx + win_size
            sub_data = data_for_plot[start_idx:end_idx]
            sub_time = np.arange(start_idx, end_idx) / 50.0
            if event_flags[j] == 3:
                plt.plot(sub_time, sub_data, '-k', linewidth=1)
                current_axis.add_patch(Rectangle((start_idx, width), win_size, height, facecolor="#FAF0BE", alpha=.2))
            elif event_flags[j] == 2:
                plt.plot(sub_time, sub_data, '-k', linewidth=1)
                current_axis.add_patch(Rectangle((start_idx, width), win_size, height, facecolor="#FF004F", alpha=1.0))
            else:
                plt.plot(sub_time, sub_data, '-k', linewidth=1)
                current_axis.add_patch(Rectangle((start_idx, width), win_size, height, facecolor="#00FFFF", alpha=.2))
        plt.savefig('./results/rawData.png')

    bad_indices = []
    for j in range(limit):
        start_idx = j * win_size
        end_idx = start_idx + win_size
        if event_flags[j] == 2 or event_flags[j] == 3:
            bad_indices.extend(range(start_idx, end_idx))
    bad_indices = np.array(bad_indices)
    valid_indices = bad_indices[(bad_indices >= 0) & (bad_indices < len(time))]
    mask = np.ones(len(data), dtype=bool)
    mask[valid_indices] = False
    filtered_data = data[mask]
    filtered_time = time[mask]
    return filtered_data, filtered_time

# Custom band-pass filter as a fallback
def custom_band_pass_filter(data, fs, lowcut, highcut, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

# Main program starts here
print('\nstart processing ...')

# Define sampling frequency
target_fs = 50  # Hz

# Load BCG and ECG data
bcg_file = r"c:\Users\maram\Downloads\code\code\trial_output\rounded_unix_bcg_synchronized.csv"
ecg_file_path = r"C:\Users\maram\Downloads\code\code\trial_output\ecg_synchronized.csv"

bcg_data = pd.read_csv(bcg_file, sep=",", header=None, skiprows=1).values
bcg_data_stream = bcg_data[:, 0].astype(float)
bcg_timestamps = bcg_data[:, 1].astype(np.int64) * 1000

print("Sample BCG timestamps (Unix ms):", bcg_timestamps[:5])
print("Sample BCG signal data:", bcg_data_stream[:5])

ecg_data = pd.read_csv(ecg_file_path, sep=",", header=None, skiprows=1).values
ecg_timestamps_str = ecg_data[:, 0]
ecg_hr = ecg_data[:, 1].astype(float)
ecg_rr = ecg_data[:, 2].astype(float)

try:
    ecg_timestamps = pd.to_datetime(ecg_timestamps_str, format='%Y/%m/%d %H:%M:%S').astype('int64') // 10**6
except ValueError as e:
    print("Error parsing ECG timestamps:", e)
    print("Please verify the timestamp format in the ECG data.")
    raise

print("Sample ECG timestamps (converted to Unix ms):", ecg_timestamps[:5])
print(f"Original ECG data points: {len(ecg_hr)}")
print(f"Original BCG data points: {len(bcg_data_stream)}")

# Detect patterns and remove segments from BCG
start_point, end_point, window_shift = 0, 500, 500
filtered_bcg, filtered_bcg_time = detect_patterns1(start_point, end_point, window_shift, bcg_data_stream, bcg_timestamps, plot=1)

filtered_bcg_seconds = np.unique(filtered_bcg_time // 1000)
ecg_seconds = ecg_timestamps // 1000
ecg_mask = np.isin(ecg_seconds, filtered_bcg_seconds)
filtered_ecg_hr = ecg_hr[ecg_mask]
filtered_ecg_timestamps = ecg_timestamps[ecg_mask]
filtered_ecg_rr = ecg_rr[ecg_mask]

# Clean data: Remove non-finite values and duplicates
mask_finite = np.isfinite(filtered_ecg_hr) & np.isfinite(filtered_ecg_timestamps)
filtered_ecg_hr = filtered_ecg_hr[mask_finite]
filtered_ecg_timestamps = filtered_ecg_timestamps[mask_finite]
_, unique_idx = np.unique(filtered_ecg_timestamps, return_index=True)
filtered_ecg_hr = filtered_ecg_hr[unique_idx]
filtered_ecg_timestamps = filtered_ecg_timestamps[unique_idx]

print(f"Cleaned ECG data points: {len(filtered_ecg_hr)}")
print(f"Cleaned ECG timestamps sample: {filtered_ecg_timestamps[:5]}")

print(f"Filtered ECG data points: {len(filtered_ecg_hr)}")
print(f"Filtered BCG data points: {len(filtered_bcg)}")
print(f"Removed {len(ecg_hr) - len(filtered_ecg_hr)} ECG data points ({(1 - len(filtered_ecg_hr)/len(ecg_hr))*100:.2f}%)")

# BCG signal extraction with custom filter
movement = band_pass_filtering(filtered_bcg, target_fs, "bcg")
movement = custom_band_pass_filter(movement, target_fs, 0.5, 5.0)  # Override with custom range

# Respiratory signal extraction with custom filter
breathing = band_pass_filtering(filtered_bcg, target_fs, "breath")
breathing = custom_band_pass_filter(breathing, target_fs, 0.1, 0.5)  # Override with custom range
breathing = remove_nonLinear_trend(breathing, 2)
breathing = savgol_filter(breathing, 11, 3)

# Apply moving average to reduce noise
movement = np.convolve(movement, np.ones(5)/5, mode='same')
breathing = np.convolve(breathing, np.ones(5)/5, mode='same')

# Wavelet transform for BCG signal
w = modwt(movement, 'db4', 5)
dc = modwtmra(w, 'db4')
wavelet_cycle = dc[4]

# Vital Signs estimation with adjusted mpd
t1, t2, window_length, window_shift = 0, 500, 500, 500
hop_size = math.floor((window_length - 1) / 2)
limit = int(math.floor(breathing.size / window_shift))

beats = vitals(t1, t2, window_shift, limit, wavelet_cycle, filtered_bcg_time, mpd=20, plot=0)  # Adjusted mpd
# Smooth beats and remove outliers
beats = savgol_filter(beats, 11, 3)
beats = beats[(beats >= 40) & (beats <= 180)]  # Physiological range

breaths = vitals(t1, t2, window_shift, limit, breathing, filtered_bcg_time, mpd=75, plot=0)
breaths = savgol_filter(breaths, 11, 3)
breaths = breaths[(breaths >= 10) & (breaths <= 60)]  # Physiological range

events = apnea_events(breathing, filtered_bcg_time, thresh=0.3)

t1, t2 = 2500, 2500 * 2
data_subplot(filtered_bcg, movement, breathing, wavelet_cycle, t1, t2)

# Resample ECG data to match BCG beats length
ecg_time_interp = np.linspace(min(filtered_ecg_timestamps), max(filtered_ecg_timestamps), len(beats))
ecg_hr_interp = interp1d(filtered_ecg_timestamps, filtered_ecg_hr, kind='linear', fill_value='extrapolate', bounds_error=False)(ecg_time_interp)
ecg_hr_interp = np.nan_to_num(ecg_hr_interp, nan=np.nanmean(ecg_hr_interp))
bcg_hr = beats[:len(ecg_hr_interp)]  # Match lengths

# Compute error metrics
mae = np.mean(np.abs(bcg_hr - ecg_hr_interp))
rmse = np.sqrt(np.mean((bcg_hr - ecg_hr_interp) ** 2))
mape = np.mean(np.abs((bcg_hr - ecg_hr_interp) / ecg_hr_interp)) * 100

print('\nHeart Rate Comparison Metrics')
print('Mean Absolute Error (MAE): ', np.around(mae, 2))
print('Root Mean Square Error (RMSE): ', np.around(rmse, 2))
print('Mean Absolute Percentage Error (MAPE): ', np.around(mape, 2), '%')

# Bland-Altman Plot
mean_hr = (bcg_hr + ecg_hr_interp) / 2
diff_hr = bcg_hr - ecg_hr_interp
mean_diff = np.mean(diff_hr)
std_diff = np.std(diff_hr)
plt.figure(figsize=(8, 6))
plt.scatter(mean_hr, diff_hr, c='blue', alpha=0.5)
plt.axhline(mean_diff, color='red', linestyle='--')
plt.axhline(mean_diff + 1.96 * std_diff, color='gray', linestyle='--')
plt.axhline(mean_diff - 1.96 * std_diff, color='gray', linestyle='--')
plt.xlabel('Mean Heart Rate (BPM)')
plt.ylabel('Difference (BCG HR - ECG HR) (BPM)')
plt.title('Bland-Altman Plot')
plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_bland_altman.png')
plt.close()

# Pearson Correlation Plot
plt.figure(figsize=(8, 6))
plt.scatter(ecg_hr_interp, bcg_hr, c='blue', alpha=0.5)
plt.plot([min(ecg_hr_interp), max(ecg_hr_interp)], [min(ecg_hr_interp), max(ecg_hr_interp)], 'r--')
plt.xlabel('ECG Heart Rate (BPM)')
plt.ylabel('BCG Heart Rate (BPM)')
plt.title('Pearson Correlation Plot')
corr, _ = pearsonr(ecg_hr_interp, bcg_hr)
plt.text(min(ecg_hr_interp), max(bcg_hr), f'Correlation: {corr:.2f}')
plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_pearson_correlation.png')
plt.close()

# Boxplot
plt.figure(figsize=(8, 6))
plt.boxplot([ecg_hr_interp, bcg_hr], labels=['ECG HR', 'BCG HR'])
plt.ylabel('Heart Rate (BPM)')
plt.title('Boxplot of ECG vs BCG Heart Rate')
plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_boxplot.png')
plt.close()

print('\nEnd processing ...')


start processing ...
Sample BCG timestamps (Unix ms): [1699125140000 1699125140000 1699125140000 1699125140000 1699125140000]
Sample BCG signal data: [ -32.7 -284.2 -844.  -242.8 -123.8]
Sample ECG timestamps (converted to Unix ms): Index([1699125140000, 1699125140000, 1699125141000, 1699125142000,
       1699125142000],
      dtype='int64')
Original ECG data points: 12617
Original BCG data points: 520580
Cleaned ECG data points: 9096
Cleaned ECG timestamps sample: Index([1699125140000, 1699125141000, 1699125142000, 1699125143000,
       1699125144000],
      dtype='int64')
Filtered ECG data points: 9096
Filtered BCG data points: 487080
Removed 3521 ECG data points (27.91%)

Apnea Information
start time :  ['07.58.19']  stop time :  ['07.58.19']

Apnea Information
start time :  ['07.58.19', '07.58.19']  stop time :  ['07.58.19', '07.58.19']

Apnea Information
start time :  ['07.58.19', '07.58.19', '07.58.19']  stop time :  ['07.58.19', '07.58.19', '07.58.19']

Apnea Information
start 

#------------------------------------ compare 

In [14]:
import math
import os
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

from band_pass_filtering import band_pass_filtering
from compute_vitals import vitals
from detect_apnea_events import apnea_events
from modwt_matlab_fft import modwt
from modwt_mra_matlab_fft import modwtmra
from remove_nonLinear_trend import remove_nonLinear_trend
from data_subplot import data_subplot
from beat_to_beat import compute_rate
 # Assuming this is the correct import for detect_patterns1

# Main program starts here
print('\nstart processing ...')

# Define sampling frequency
target_fs = 50  # Hz, as the data is already resampled to 50 Hz

# Load BCG and ECG data
bcg_file = r"c:\Users\maram\Downloads\code\code\trial_output\rounded_unix_bcg_synchronized.csv"
ecg_file_path = r"C:\Users\maram\Downloads\code\code\trial_output\ecg_synchronized.csv"

# Load BCG data
bcg_data = pd.read_csv(bcg_file, sep=",", header=None, skiprows=1).values
bcg_data_stream = bcg_data[:, 0].astype(float)  # First column is the signal data (amplitude)
bcg_timestamps = bcg_data[:, 1].astype(np.int64) * 1000  # Convert seconds to milliseconds

# Debug: Print a few BCG timestamps to verify format
print("Sample BCG timestamps (Unix ms):", bcg_timestamps[:5])
print("Sample BCG signal data:", bcg_data_stream[:5])

# Load ECG data (assuming format: Timestamp, Heart Rate, RR Interval in seconds)
ecg_data = pd.read_csv(ecg_file_path, sep=",", header=None, skiprows=1).values
ecg_timestamps_str = ecg_data[:, 0]  # Timestamps as strings
ecg_hr = ecg_data[:, 1].astype(float)  # Heart Rate (reference HR)
ecg_rr = ecg_data[:, 2].astype(float)  # RR Interval in seconds

# Convert ECG timestamps to Unix timestamps (milliseconds)
try:
    ecg_timestamps = pd.to_datetime(ecg_timestamps_str, format='%Y/%m/%d %H:%M:%S').astype('int64') // 10**6  # Convert to ms
except ValueError as e:
    print("Error parsing ECG timestamps:", e)
    print("Please verify the timestamp format in the ECG data.")
    raise

# Debug: Print converted timestamps
print("Sample ECG timestamps (converted to Unix ms):", ecg_timestamps[:5])
print(f"Original ECG data points: {len(ecg_hr)}")
print(f"Original BCG data points: {len(bcg_data_stream)}")

# Detect patterns and remove segments from BCG
start_point, end_point, window_shift = 0, 500, 500
filtered_bcg, filtered_bcg_time = detect_patterns1(start_point, end_point, window_shift, bcg_data_stream, bcg_timestamps, plot=1)

# Find unique seconds in filtered BCG timestamps (converting from ms to s)
filtered_bcg_seconds = np.unique(filtered_bcg_time // 1000)

# Create a mask for ECG data that matches with valid BCG seconds
ecg_seconds = ecg_timestamps // 1000
ecg_mask = np.isin(ecg_seconds, filtered_bcg_seconds)

# Apply the mask to filter ECG data
filtered_ecg_hr = ecg_hr[ecg_mask]
filtered_ecg_timestamps = ecg_timestamps[ecg_mask]
filtered_ecg_rr = ecg_rr[ecg_mask]

print(f"Filtered ECG data points: {len(filtered_ecg_hr)}")
print(f"Filtered BCG data points: {len(filtered_bcg)}")
print(f"Removed {len(ecg_hr) - len(filtered_ecg_hr)} ECG data points ({(1 - len(filtered_ecg_hr)/len(ecg_hr))*100:.2f}%)")

# BCG signal extraction
movement = band_pass_filtering(filtered_bcg, target_fs, "bcg")

# Respiratory signal extraction
breathing = band_pass_filtering(filtered_bcg, target_fs, "breath")
breathing = remove_nonLinear_trend(breathing, 3)
breathing = savgol_filter(breathing, 11, 3)

# Wavelet transform for BCG signal
w = modwt(movement, 'bior3.9', 4)
dc = modwtmra(w, 'bior3.9')
wavelet_cycle = dc[4]

# Vital Signs estimation (10 seconds window)
t1, t2, window_length, window_shift = 0, 500, 500, 500
hop_size = math.floor((window_length - 1) / 2)
limit = int(math.floor(breathing.size / window_shift))

# Compute Heart Rate from BCG (using J-peaks)
beats = vitals(t1, t2, window_shift, limit, wavelet_cycle, filtered_bcg_time, mpd=15, plot=0)
print('\nHeart Rate Information (BCG)')
print('Minimum pulse : ', np.around(np.min(beats)))
print('Maximum pulse : ', np.around(np.max(beats)))
print('Average pulse : ', np.around(np.mean(beats)))
print('Number of BCG windows :', len(beats))

# Compute number of ECG windows
ecg_windows = []
for i in range(limit):
    # Define the time window (in milliseconds)
    window_start = filtered_bcg_time[0] + i * window_shift * 1000 / target_fs  # Convert samples to ms
    window_end = window_start + window_length * 1000 / target_fs  # Window length in ms
    
    # Find ECG data points within this time window
    window_mask = (filtered_ecg_timestamps >= window_start) & (filtered_ecg_timestamps < window_end)
    if np.any(window_mask):
        # If there is at least one ECG data point in the window, count it as a valid window
        # Use the average heart rate in this window (or first HR if multiple points)
        window_ecg_hr = filtered_ecg_hr[window_mask]
        if len(window_ecg_hr) > 0:
            ecg_windows.append(np.mean(window_ecg_hr))  # Store mean HR for the window

print('\nECG Window Information')
print('Number of ECG windows :', len(ecg_windows))
if len(ecg_windows) > 0:
    print('Minimum ECG HR (windowed) : ', np.around(np.min(ecg_windows)))
    print('Maximum ECG HR (windowed) : ', np.around(np.max(ecg_windows)))
    print('Average ECG HR (windowed) : ', np.around(np.mean(ecg_windows)))

# Breathing Rate with adjusted mpd
breaths = vitals(t1, t2, window_shift, limit, breathing, filtered_bcg_time, mpd=75, plot=0)
print('\nRespiratory Rate Information')
print('Minimum breathing : ', np.around(np.min(breaths)))
print('Maximum breathing : ', np.around(np.max(breaths)))
print('Average breathing : ', np.around(np.mean(breaths)))

# Apnea Events Detection
thresh = 0.3
events = apnea_events(breathing, filtered_bcg_time, thresh=thresh)

# Plot Vitals Example
t1, t2 = 2500, 2500 * 2
data_subplot(filtered_bcg, movement, breathing, wavelet_cycle, t1, t2)

# Compare BCG-derived HR with ECG-derived HR
min_length = min(len(beats), len(filtered_ecg_hr))
bcg_hr = beats[:min_length]
ecg_hr_ref = filtered_ecg_hr[:min_length]

# Compute error metrics
mae = np.mean(np.abs(bcg_hr - ecg_hr_ref))
rmse = np.sqrt(np.mean((bcg_hr - ecg_hr_ref) ** 2))
mape = np.mean(np.abs((bcg_hr - ecg_hr_ref) / ecg_hr_ref)) * 100

print('\nHeart Rate Comparison Metrics')
print('Mean Absolute Error (MAE): ', np.around(mae, 2))
print('Root Mean Square Error (RMSE): ', np.around(rmse, 2))
print('Mean Absolute Percentage Error (MAPE): ', np.around(mape, 2), '%')

# Bland-Altman Plot
mean_hr = (bcg_hr + ecg_hr_ref) / 2
diff_hr = bcg_hr - ecg_hr_ref
mean_diff = np.mean(diff_hr)
std_diff = np.std(diff_hr)
plt.figure(figsize=(8, 6))
plt.scatter(mean_hr, diff_hr, c='blue', alpha=0.5)
plt.axhline(mean_diff, color='red', linestyle='--')
plt.axhline(mean_diff + 1.96 * std_diff, color='gray', linestyle='--')
plt.axhline(mean_diff - 1.96 * std_diff, color='gray', linestyle='--')
plt.xlabel('Mean Heart Rate (BPM)')
plt.ylabel('Difference (BCG HR - ECG HR) (BPM)')
plt.title('Bland-Altman Plot')
plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_bland_altman.png')
plt.close()

# Pearson Correlation Plot
plt.figure(figsize=(8, 6))
plt.scatter(ecg_hr_ref, bcg_hr, c='blue', alpha=0.5)
plt.plot([min(ecg_hr_ref), max(ecg_hr_ref)], [min(ecg_hr_ref), max(ecg_hr_ref)], 'r--')
plt.xlabel('ECG Heart Rate (BPM)')
plt.ylabel('BCG Heart Rate (BPM)')
plt.title('Pearson Correlation Plot')
corr, _ = pearsonr(ecg_hr_ref, bcg_hr)
plt.text(min(ecg_hr_ref), max(bcg_hr), f'Correlation: {corr:.2f}')
plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_pearson_correlation.png')
plt.close()

# Boxplot
plt.figure(figsize=(8, 6))
plt.boxplot([ecg_hr_ref, bcg_hr], labels=['ECG HR', 'BCG HR'])
plt.ylabel('Heart Rate (BPM)')
plt.title('Boxplot of ECG vs BCG Heart Rate')
plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_boxplot.png')
plt.close()

print('\nEnd processing ...')


start processing ...
Sample BCG timestamps (Unix ms): [1699125140000 1699125140000 1699125140000 1699125140000 1699125140000]
Sample BCG signal data: [ -32.7 -284.2 -844.  -242.8 -123.8]
Sample ECG timestamps (converted to Unix ms): Index([1699125140000, 1699125140000, 1699125141000, 1699125142000,
       1699125142000],
      dtype='int64')
Original ECG data points: 12617
Original BCG data points: 520580
Filtered ECG data points: 11896
Filtered BCG data points: 487080
Removed 721 ECG data points (5.71%)

Heart Rate Information (BCG)
Minimum pulse :  60.0
Maximum pulse :  113.0
Average pulse :  87.0
Number of BCG windows : 974

ECG Window Information
Number of ECG windows : 954
Minimum ECG HR (windowed) :  61.0
Maximum ECG HR (windowed) :  107.0
Average ECG HR (windowed) :  79.0

Respiratory Rate Information
Minimum breathing :  17.0
Maximum breathing :  40.0
Average breathing :  29.0

Apnea Information
start time :  ['07.58.19']  stop time :  ['07.58.19']

Apnea Information
start tim

In [ ]:
import math
import os
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

from band_pass_filtering import band_pass_filtering
from compute_vitals import vitals
from detect_apnea_events import apnea_events
from modwt_matlab_fft import modwt
from modwt_mra_matlab_fft import modwtmra
from remove_nonLinear_trend import remove_nonLinear_trend
from data_subplot import data_subplot
from beat_to_beat import compute_rate
 # Assuming this is the correct import

# Main program starts here
print('\nstart processing ...')

# Define sampling frequency
target_fs = 50  # Hz, as the data is already resampled to 50 Hz

# Load BCG and ECG data
bcg_file = r"c:\Users\maram\Downloads\code\code\trial_output\rounded_unix_bcg_synchronized.csv"
ecg_file_path = r"C:\Users\maram\Downloads\code\code\trial_output\ecg_synchronized.csv"

# Load BCG data
bcg_data = pd.read_csv(bcg_file, sep=",", header=None, skiprows=1).values
bcg_data_stream = bcg_data[:, 0].astype(float)  # First column is the signal data (amplitude)
bcg_timestamps = bcg_data[:, 1].astype(np.int64) * 1000  # Convert seconds to milliseconds

# Debug: Print a few BCG timestamps to verify format
print("Sample BCG timestamps (Unix ms):", bcg_timestamps[:5])
print("Sample BCG signal data:", bcg_data_stream[:5])

# Load ECG data (assuming format: Timestamp, Heart Rate, RR Interval in seconds)
ecg_data = pd.read_csv(ecg_file_path, sep=",", header=None, skiprows=1).values
ecg_timestamps_str = ecg_data[:, 0]  # Timestamps as strings
ecg_hr = ecg_data[:, 1].astype(float)  # Heart Rate (reference HR)
ecg_rr = ecg_data[:, 2].astype(float)  # RR Interval in seconds

# Convert ECG timestamps to Unix timestamps (milliseconds)
try:
    ecg_timestamps = pd.to_datetime(ecg_timestamps_str, format='%Y/%m/%d %H:%M:%S').astype('int64') // 10**6  # Convert to ms
except ValueError as e:
    print("Error parsing ECG timestamps:", e)
    print("Please verify the timestamp format in the ECG data.")
    raise

# Debug: Print converted timestamps
print("Sample ECG timestamps (converted to Unix ms):", ecg_timestamps[:5])
print(f"Original ECG data points: {len(ecg_hr)}")
print(f"Original BCG data points: {len(bcg_data_stream)}")

# Detect patterns and remove segments from BCG
start_point, end_point, window_shift = 0, 500, 500
filtered_bcg, filtered_bcg_time = detect_patterns1(start_point, end_point, window_shift, bcg_data_stream, bcg_timestamps, plot=1)

# Find unique seconds in filtered BCG timestamps (converting from ms to s)
filtered_bcg_seconds = np.unique(filtered_bcg_time // 1000)

# Create a mask for ECG data that matches with valid BCG seconds
ecg_seconds = ecg_timestamps // 1000
ecg_mask = np.isin(ecg_seconds, filtered_bcg_seconds)

# Apply the mask to filter ECG data
filtered_ecg_hr = ecg_hr[ecg_mask]
filtered_ecg_timestamps = ecg_timestamps[ecg_mask]
filtered_ecg_rr = ecg_rr[ecg_mask]

print(f"Filtered ECG data points: {len(filtered_ecg_hr)}")
print(f"Filtered BCG data points: {len(filtered_bcg)}")
print(f"Removed {len(ecg_hr) - len(filtered_ecg_hr)} ECG data points ({(1 - len(filtered_ecg_hr)/len(ecg_hr))*100:.2f}%)")

# BCG signal extraction
movement = band_pass_filtering(filtered_bcg, target_fs, "bcg")

# Respiratory signal extraction
breathing = band_pass_filtering(filtered_bcg, target_fs, "breath")
breathing = remove_nonLinear_trend(breathing, 3)
breathing = savgol_filter(breathing, 11, 3)

# Wavelet transform for BCG signal
w = modwt(movement, 'bior3.9', 4)
dc = modwtmra(w, 'bior3.9')
wavelet_cycle = dc[4]

# Vital Signs estimation (10 seconds window)
t1, t2, window_length, window_shift = 0, 500, 500, 500
hop_size = math.floor((window_length - 1) / 2)
limit = int(math.floor(breathing.size / window_shift))

# Determine valid windows where both BCG and ECG data exist
valid_windows = []
bcg_windows = []
ecg_windows = []
for i in range(limit):
    # Define the time window (in milliseconds)
    window_start = filtered_bcg_time[0] + i * window_shift * 1000 / target_fs  # Convert samples to ms
    window_end = window_start + window_length * 1000 / target_fs  # Window length in ms
    
    # Check for BCG data in the window (slice the signal)
    bcg_indices = (filtered_bcg_time >= window_start) & (filtered_bcg_time < window_end)
    if not np.any(bcg_indices):
        continue  # Skip if no BCG data
    
    # Check for ECG data in the window
    ecg_window_mask = (filtered_ecg_timestamps >= window_start) & (filtered_ecg_timestamps < window_end)
    if not np.any(ecg_window_mask):
        continue  # Skip if no ECG data
    
    # Valid window: both BCG and ECG data exist
    valid_windows.append(i)
    
    # Compute BCG heart rate for this window (will be processed later in vitals)
    bcg_windows.append(i)
    
    # Compute ECG heart rate for this window
    window_ecg_hr = filtered_ecg_hr[ecg_window_mask]
    if len(window_ecg_hr) > 0:
        ecg_windows.append(np.mean(window_ecg_hr))  # Store mean HR for the window

# Compute Heart Rate from BCG only for valid windows
beats = []
for i in valid_windows:
    # Compute the start index for the BCG signal
    start_idx = i * window_shift
    end_idx = min(start_idx + window_length, len(wavelet_cycle))
    if end_idx <= start_idx:
        continue
    window_signal = wavelet_cycle[start_idx:end_idx]
    window_time = filtered_bcg_time[start_idx:end_idx]
    if len(window_signal) < 2 or len(window_time) < 2:
        continue
    # Call vitals for this specific window
    window_beats = vitals(t1, t2, window_shift, 1, window_signal, window_time, mpd=15, plot=0)
    if len(window_beats) > 0:
        beats.append(window_beats[0])  # Take the first (or only) HR value

beats = np.array(beats)
print('\nHeart Rate Information (BCG)')
print('Minimum pulse : ', np.around(np.min(beats)) if len(beats) > 0 else 'N/A')
print('Maximum pulse : ', np.around(np.max(beats)) if len(beats) > 0 else 'N/A')
print('Average pulse : ', np.around(np.mean(beats)) if len(beats) > 0 else 'N/A')
print('Number of BCG windows :', len(beats))

# ECG Window Information
print('\nECG Window Information')
print('Number of ECG windows :', len(ecg_windows))
if len(ecg_windows) > 0:
    print('Minimum ECG HR (windowed) : ', np.around(np.min(ecg_windows)))
    print('Maximum ECG HR (windowed) : ', np.around(np.max(ecg_windows)))
    print('Average ECG HR (windowed) : ', np.around(np.mean(ecg_windows)))

# Breathing Rate with adjusted mpd for valid windows
breaths = []
for i in valid_windows:
    start_idx = i * window_shift
    end_idx = min(start_idx + window_length, len(breathing))
    if end_idx <= start_idx:
        continue
    window_signal = breathing[start_idx:end_idx]
    window_time = filtered_bcg_time[start_idx:end_idx]
    if len(window_signal) < 2 or len(window_time) < 2:
        continue
    window_breaths = vitals(t1, t2, window_shift, 1, window_signal, window_time, mpd=75, plot=0)
    if len(window_breaths) > 0:
        breaths.append(window_breaths[0])

breaths = np.array(breaths)
print('\nRespiratory Rate Information')
print('Minimum breathing : ', np.around(np.min(breaths)) if len(breaths) > 0 else 'N/A')
print('Maximum breathing : ', np.around(np.max(breaths)) if len(breaths) > 0 else 'N/A')
print('Average breathing : ', np.around(np.mean(breaths)) if len(breaths) > 0 else 'N/A')

# Apnea Events Detection
thresh = 0.3
events = apnea_events(breathing, filtered_bcg_time, thresh=thresh)

# Plot Vitals Example
t1, t2 = 2500, 2500 * 2
data_subplot(filtered_bcg, movement, breathing, wavelet_cycle, t1, t2)

# Compare BCG-derived HR with ECG-derived HR
bcg_hr = np.array(beats)
ecg_hr_ref = np.array(ecg_windows)

# Compute error metrics
mae = np.mean(np.abs(beats - ecg_windows)) if len(beats) > 0 else 'N/A'
rmse = np.sqrt(np.mean((beats -ecg_windows) ** 2)) if len(beats) > 0 else 'N/A'
mape = np.mean(np.abs((beats - ecg_windows) / ecg_windows)) * 100 if len(beats) > 0 and not np.any(ecg_windows== 0) else 'N/A'

print('\nHeart Rate Comparison Metrics')
print('Mean Absolute Error (MAE): ', np.around(mae, 2) if isinstance(mae, float) else mae)
print('Root Mean Square Error (RMSE): ', np.around(rmse, 2) if isinstance(rmse, float) else rmse)
print('Mean Absolute Percentage Error (MAPE): ', np.around(mape, 2), '%' if isinstance(mape, float) else mape)

# Bland-Altman Plot
if len(bcg_hr) > 0:
    mean_hr = (bcg_hr + ecg_hr_ref) / 2
    diff_hr = bcg_hr - ecg_hr_ref
    mean_diff = np.mean(diff_hr)
    std_diff = np.std(diff_hr)
    plt.figure(figsize=(8, 6))
    plt.scatter(mean_hr, diff_hr, c='blue', alpha=0.5)
    plt.axhline(mean_diff, color='red', linestyle='--')
    plt.axhline(mean_diff + 1.96 * std_diff, color='gray', linestyle='--')
    plt.axhline(mean_diff - 1.96 * std_diff, color='gray', linestyle='--')
    plt.xlabel('Mean Heart Rate (BPM)')
    plt.ylabel('Difference (BCG HR - ECG HR) (BPM)')
    plt.title('Bland-Altman Plot')
    plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_bland_altman.png')
    plt.close()

# Pearson Correlation Plot
if len(bcg_hr) > 0:
    plt.figure(figsize=(8, 6))
    plt.scatter(import math
import os
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

from band_pass_filtering import band_pass_filtering
from compute_vitals import vitals
from detect_apnea_events import apnea_events
from modwt_matlab_fft import modwt
from modwt_mra_matlab_fft import modwtmra
from remove_nonLinear_trend import remove_nonLinear_trend
from data_subplot import data_subplot
from beat_to_beat import compute_rate
from detect_patterns import detect_patterns1  # Assuming this is the correct import

# Main program starts here
print('\nstart processing ...')

# Define sampling frequency
target_fs = 50  # Hz, as the data is already resampled to 50 Hz

# Load BCG and ECG data
bcg_file = r"c:\Users\maram\Downloads\code\code\trial_output\rounded_unix_bcg_synchronized.csv"
ecg_file_path = r"C:\Users\maram\Downloads\code\code\trial_output\ecg_synchronized.csv"

# Load BCG data
bcg_data = pd.read_csv(bcg_file, sep=",", header=None, skiprows=1).values
bcg_data_stream = bcg_data[:, 0].astype(float)  # First column is the signal data (amplitude)
bcg_timestamps = bcg_data[:, 1].astype(np.int64) * 1000  # Convert seconds to milliseconds

# Debug: Print a few BCG timestamps to verify format
print("Sample BCG timestamps (Unix ms):", bcg_timestamps[:5])
print("Sample BCG signal data:", bcg_data_stream[:5])

# Load ECG data (assuming format: Timestamp, Heart Rate, RR Interval in seconds)
ecg_data = pd.read_csv(ecg_file_path, sep=",", header=None, skiprows=1).values
ecg_timestamps_str = ecg_data[:, 0]  # Timestamps as strings
ecg_hr = ecg_data[:, 1].astype(float)  # Heart Rate (reference HR)
ecg_rr = ecg_data[:, 2].astype(float)  # RR Interval in seconds

# Convert ECG timestamps to Unix timestamps (milliseconds)
try:
    ecg_timestamps = pd.to_datetime(ecg_timestamps_str, format='%Y/%m/%d %H:%M:%S').astype('int64') // 10**6  # Convert to ms
except ValueError as e:
    print("Error parsing ECG timestamps:", e)
    print("Please verify the timestamp format in the ECG data.")
    raise

# Debug: Print converted timestamps
print("Sample ECG timestamps (converted to Unix ms):", ecg_timestamps[:5])
print(f"Original ECG data points: {len(ecg_hr)}")
print(f"Original BCG data points: {len(bcg_data_stream)}")

# Detect patterns and remove segments from BCG
start_point, end_point, window_length = 0, 500, 500
window_shift = 250  # 50% overlap for better resolution
filtered_bcg, filtered_bcg_time = detect_patterns1(start_point, end_point, window_length, bcg_data_stream, bcg_timestamps, plot=1)

# Find unique seconds in filtered BCG timestamps (converting from ms to s)
filtered_bcg_seconds = np.unique(filtered_bcg_time // 1000)

# Create a mask for ECG data that matches with valid BCG seconds
ecg_seconds = ecg_timestamps // 1000
ecg_mask = np.isin(ecg_seconds, filtered_bcg_seconds)

# Apply the mask to filter ECG data
filtered_ecg_hr = ecg_hr[ecg_mask]
filtered_ecg_timestamps = ecg_timestamps[ecg_mask]
filtered_ecg_rr = ecg_rr[ecg_mask]

print(f"Filtered ECG data points: {len(filtered_ecg_hr)}")
print(f"Filtered BCG data points: {len(filtered_bcg)}")
print(f"Removed {len(ecg_hr) - len(filtered_ecg_hr)} ECG data points ({(1 - len(filtered_ecg_hr)/len(ecg_hr))*100:.2f}%)")

# BCG signal extraction with optimized frequency range
movement = band_pass_filtering(filtered_bcg, target_fs, "bcg", lowcut=0.8, highcut=2.5, order=4)

# Respiratory signal extraction
breathing = band_pass_filtering(filtered_bcg, target_fs, "breath")
breathing = remove_nonLinear_trend(breathing, 3)
breathing = savgol_filter(breathing, 11, 3)

# Wavelet transform with thresholding
w = modwt(movement, 'db4', 4)  # Switch to 'db4' wavelet
# Soft thresholding on wavelet coefficients
for level in range(w.shape[0]):
    thresh = np.std(w[level]) * np.sqrt(2 * np.log(len(w[level]))) / 2  # Universal threshold
    w[level] = np.where(np.abs(w[level]) > thresh, w[level], 0)
dc = modwtmra(w, 'db4')
wavelet_cycle = dc[3] + dc[4]  # Combine levels 3 and 4 for cardiac signal

# Vital Signs estimation
t1, t2 = 0, 500
hop_size = math.floor((window_length - 1) / 2)
limit = int(math.floor((len(breathing) - window_length) / window_shift)) + 1

# Determine valid windows where both BCG and ECG data exist
valid_windows = []
bcg_windows = []
ecg_windows = []
for i in range(limit):
    # Define the time window (in milliseconds)
    window_start = filtered_bcg_time[0] + i * window_shift * 1000 / target_fs
    window_end = window_start + window_length * 1000 / target_fs
    
    # Check for BCG data in the window
    bcg_indices = (filtered_bcg_time >= window_start) & (filtered_bcg_time < window_end)
    if not np.any(bcg_indices):
        continue
    
    # Check for ECG data in the window
    ecg_window_mask = (filtered_ecg_timestamps >= window_start) & (filtered_ecg_timestamps < window_end)
    if not np.any(ecg_window_mask):
        continue
    
    # Valid window: both BCG and ECG data exist
    valid_windows.append(i)
    
    # Store window index for BCG
    bcg_windows.append(i)
    
    # Compute ECG heart rate for this window
    window_ecg_hr = filtered_ecg_hr[ecg_window_mask]
    if len(window_ecg_hr) > 0:
        ecg_windows.append(np.mean(window_ecg_hr))

# Compute Heart Rate from BCG only for valid windows
beats = []
for i in valid_windows:
    start_idx = i * window_shift
    end_idx = min(start_idx + window_length, len(wavelet_cycle))
    if end_idx <= start_idx:
        continue
    window_signal = wavelet_cycle[start_idx:end_idx]
    window_time = filtered_bcg_time[start_idx:end_idx]
    if len(window_signal) < 2 or len(window_time) < 2:
        continue
    # Dynamic MPD based on expected heart rate (60–120 BPM)
    mpd = int(0.5 * target_fs)  # ~60 BPM (1 second = 50 samples)
    window_beats = vitals(t1, t2, window_shift, 1, window_signal, window_time, mpd=mpd, plot=0)
    if len(window_beats) > 0:
        beats.append(window_beats[0])

beats = np.array(beats)
# Apply Savitzky-Golay smoothing to BCG heart rates
if len(beats) > 5:  # Ensure enough points for smoothing
    beats = savgol_filter(beats, window_length=5, polyorder=2)

# Outlier removal (heart rates outside 40–150 BPM)
valid_indices = (beats >= 40) & (beats <= 150) & (np.array(ecg_windows) >= 40) & (np.array(ecg_windows) <= 150)
beats = beats[valid_indices]
ecg_windows = np.array(ecg_windows)[valid_indices]

print('\nHeart Rate Information (BCG)')
print('Minimum pulse : ', np.around(np.min(beats)) if len(beats) > 0 else 'N/A')
print('Maximum pulse : ', np.around(np.max(beats)) if len(beats) > 0 else 'N/A')
print('Average pulse : ', np.around(np.mean(beats)) if len(beats) > 0 else 'N/A')
print('Number of BCG windows :', len(beats))

# ECG Window Information
print('\nECG Window Information')
print('Number of ECG windows :', len(ecg_windows))
if len(ecg_windows) > 0:
    print('Minimum ECG HR (windowed) : ', np.around(np.min(ecg_windows)))
    print('Maximum ECG HR (windowed) : ', np.around(np.max(ecg_windows)))
    print('Average ECG HR (windowed) : ', np.around(np.mean(ecg_windows)))

# Breathing Rate with adjusted mpd for valid windows
breaths = []
for i in valid_windows:
    start_idx = i * window_shift
    end_idx = min(start_idx + window_length, len(breathing))
    if end_idx <= start_idx:
        continue
    window_signal = breathing[start_idx:end_idx]
    window_time = filtered_bcg_time[start_idx:end_idx]
    if len(window_signal) < 2 or len(window_time) < 2:
        continue
    window_breaths = vitals(t1, t2, window_shift, 1, window_signal, window_time, mpd=75, plot=0)
    if len(window_breaths) > 0:
        breaths.append(window_breaths[0])

breaths = np.array(breaths)
print('\nRespiratory Rate Information')
print('Minimum breathing : ', np.around(np.min(breaths)) if len(breaths) > 0 else 'N/A')
print('Maximum breathing : ', np.around(np.max(breaths)) if len(breaths) > 0 else 'N/A')
print('Average breathing : ', np.around(np.mean(breaths)) if len(breaths) > 0 else 'N/A')

# Apnea Events Detection
thresh = 0.3
events = apnea_events(breathing, filtered_bcg_time, thresh=thresh)

# Plot Vitals Example
t1, t2 = 2500, 2500 * 2
data_subplot(filtered_bcg, movement, breathing, wavelet_cycle, t1, t2)

# Compare BCG-derived HR with ECG-derived HR
bcg_hr = np.array(beats)
ecg_hr_ref = np.array(ecg_windows)

# Compute error metrics
mae = np.mean(np.abs(bcg_hr - ecg_hr_ref)) if len(bcg_hr) > 0 else 'N/A'
rmse = np.sqrt(np.mean((bcg_hr - ecg_hr_ref) ** 2)) if len(bcg_hr) > 0 else 'N/A'
mape = np.mean(np.abs((bcg_hr - ecg_hr_ref) / ecg_hr_ref)) * 100 if len(bcg_hr) > 0 and not np.any(ecg_hr_ref == 0) else 'N/A'

print('\nHeart Rate Comparison Metrics')
print('Mean Absolute Error (MAE): ', np.around(mae, 2) if isinstance(mae, float) else mae)
print('Root Mean Square Error (RMSE): ', np.around(rmse, 2) if isinstance(rmse, float) else rmse)
print('Mean Absolute Percentage Error (MAPE): ', np.around(mape, 2), '%' if isinstance(mape, float) else mape)

# Bland-Altman Plot
if len(bcg_hr) > 0:
    mean_hr = (bcg_hr + ecg_hr_ref) / 2
    diff_hr = bcg_hr - ecg_hr_ref
    mean_diff = np.mean(diff_hr)
    std_diff = np.std(diff_hr)
    plt.figure(figsize=(8, 6))
    plt.scatter(mean_hr, diff_hr, c='blue', alpha=0.5)
    plt.axhline(mean_diff, color='red', linestyle='--')
    plt.axhline(mean_diff + 1.96 * std_diff, color='gray', linestyle='--')
    plt.axhline(mean_diff - 1.96 * std_diff, color='gray', linestyle='--')
    plt.xlabel('Mean Heart Rate (BPM)')
    plt.ylabel('Difference (BCG HR - ECG HR) (BPM)')
    plt.title('Bland-Altman Plot')
    plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_bland_altman.png')
    plt.close()

# Pearson Correlation Plot
if len(bcg_hr) > 0:
    plt.figure(figsize=(8, 6))
    plt.scatter(ecg_hr_ref, bcg_hr, c='blue', alpha=0.5)
    plt.plot([min(ecg_hr_ref), max(ecg_hr_ref)], [min(ecg_hr_ref), max(ecg_hr_ref)], 'r--')
    plt.xlabel('ECG Heart Rate (BPM)')
    plt.ylabel('BCG Heart Rate (BPM)')
    plt.title('Pearson Correlation Plot')
    corr, _ = pearsonr(ecg_hr_ref, bcg_hr)
    plt.text(min(ecg_hr_ref), max(bcg_hr), f'Correlation: {corr:.2f}')
    plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_pearson_correlation.png')
    plt.close()

# Boxplot
if len(bcg_hr) > 0:
    plt.figure(figsize=(8, 6))
    plt.boxplot([ecg_hr_ref, bcg_hr], labels=['ECG HR', 'BCG HR'])
    plt.ylabel('Heart Rate (BPM)')
    plt.title('Boxplot of ECG vs BCG Heart Rate')
    plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_boxplot.png')
    plt.close()

print('\nEnd processing ...'), bcg_hr, c='blue', alpha=0.5)
    plt.plot([min(ecg_hr_ref), max(ecg_hr_ref)], [min(ecg_hr_ref), max(ecg_hr_ref)], 'r--')
    plt.xlabel('ECG Heart Rate (BPM)')
    plt.ylabel('BCG Heart Rate (BPM)')
    plt.title('Pearson Correlation Plot')
    corr, _ = pearsonr(ecg_hr_ref, bcg_hr)
    plt.text(min(ecg_hr_ref), max(bcg_hr), f'Correlation: {corr:.2f}')
    plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_pearson_correlation.png')
    plt.close()

# Boxplot
if len(bcg_hr) > 0:
    plt.figure(figsize=(8, 6))
    plt.boxplot([ecg_hr_ref, bcg_hr], labels=['ECG HR', 'BCG HR'])
    plt.ylabel('Heart Rate (BPM)')
    plt.title('Boxplot of ECG vs BCG Heart Rate')
    plt.savefig(r'C:\Users\maram\Desktop\New folder (5)\code\results\2_boxplot.png')
    plt.close()

print('\nEnd processing ...')


start processing ...
Sample BCG timestamps (Unix ms): [1699125140000 1699125140000 1699125140000 1699125140000 1699125140000]
Sample BCG signal data: [ -32.7 -284.2 -844.  -242.8 -123.8]
Sample ECG timestamps (converted to Unix ms): Index([1699125140000, 1699125140000, 1699125141000, 1699125142000,
       1699125142000],
      dtype='int64')
Original ECG data points: 12617
Original BCG data points: 520580
Filtered ECG data points: 11896
Filtered BCG data points: 487080
Removed 721 ECG data points (5.71%)

Heart Rate Information (BCG)
Minimum pulse :  60.0
Maximum pulse :  113.0
Average pulse :  87.0
Number of BCG windows : 954

ECG Window Information
Number of ECG windows : 954
Minimum ECG HR (windowed) :  61.0
Maximum ECG HR (windowed) :  107.0
Average ECG HR (windowed) :  79.0

Respiratory Rate Information
Minimum breathing :  17.0
Maximum breathing :  40.0
Average breathing :  29.0

Apnea Information
start time :  ['07.58.19']  stop time :  ['07.58.19']

Apnea Information
start tim